 Esta notebook tomará un video, moviendo mano de la derecha y luego la izquierda, que vas a grabar y luego etiquetar. Paso siguiente, generas un modelo, para luego utilizar como predicción.

In [1]:
!pip install mediapipe opencv-python

In [1]:
import cv2
import mediapipe as mp
import numpy as np

In [2]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [13]:
# Capturar video: es solo prueba de que funciona. No realiza mas que mostrar.
# Para salir pulsa la tecla "q"
cap = cv2.VideoCapture(0)
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        #Recolorear 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Detectar
        results = pose.process(image) 
        
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                            mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                            mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                            )
        
        cv2.imshow('Raw Webcam Feed', image)
        
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 

## Grabar Video


In [14]:
# import time
VIDEO_PATH = 'training_data/push_ups_2.avi'
EXPORT_PATH = 'push_ups_2.csv'
MODEL_PATH = 'push_ups.pkl'

In [100]:
#1. Grabar Video. Para terminar de grabar pulsa la tecla "q"
cap = cv2.VideoCapture(0)
alto = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
ancho = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
fps = cap.get(cv2.CAP_PROP_FPS)

videoWriter = cv2.VideoWriter(VIDEO_PATH, cv2.VideoWriter_fourcc('P','I','M','1'), fps, (int(ancho), int(alto)))

# time.sleep(5)

while cap.isOpened():
    ret, frame = cap.read()
    
    try:
        cv2.imshow('Imagen', frame)
        videoWriter.write(frame)
    except Exception as e:
        break
    
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break
cap.release()
videoWriter.release()
cv2.destroyAllWindows()

# Capturar puntos de referencia y exportar a CSV


Esta sección crea el archivo, colocas los titulos y arma un primera estructura

In [15]:
import csv
import os
import numpy as np
from matplotlib import pyplot as plt

In [16]:
landmarks = ['class']
for val in range(1,33+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val), 'v{}'.format(val)]

In [17]:
with open(EXPORT_PATH, mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [18]:
def export_landmark(results, action):
    try:
        keypoints = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
        keypoints.insert(0, action)

        with open(EXPORT_PATH, mode='a', newline='') as f:
            csv_writer = csv.writer(f, delimiter=',',quotechar='"',quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)
    except Exception as e:
        pass


In [19]:
export_landmark(results, 'neutro' )

## Etiquetado y grabo con coordenadas

A medida que se mira el video y tenemos las coordenadas, vamos etiquetando manualmente. A medida que lo etiquetamos va guardando en un archivo csv.
Para equitar pulsa la tecla "d" para derecha y lo mantienes pulsa mientras miras el video la posición. Con tecla "i" para izquierda y lo mantienes pulsado mientras sea esta situación.
Si ninguna de las dos, posición neutra, pulsa la tecla "n".
Para salir del video y no produzca error al final, pulsamos la tecla "q".


In [20]:
cap = cv2.VideoCapture(VIDEO_PATH)    

with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:

    while cap.isOpened():
        ret, frame = cap.read()

        #Recolorear 
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        
        #Detectar
        results = pose.process(image) 

        #Recolorear el fondo de la imagen para para renderizar
        image.flags.writeable = True
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        k = cv2.waitKey(1)
        ## export_landmark(results, k)
        ## con las teclas d (100) y u (117) etiqueto en linea
        if k == ord('e'):
            export_landmark(results, 'extendido')
        if k == ord('n'):
            export_landmark(results, 'neutro')            
        if k == ord('f'):
            export_landmark(results, 'flexionado')

        cv2.imshow('Video Grabado', image)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

error: OpenCV(4.7.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


Si queda con error el anterior codigo, lo cerramos con el codigo que sigue.

In [21]:
cap.release()
cv2.destroyAllWindows()

## Entrenar un modelo usando Scikit Learn


In [23]:
!pip install scikit-learn

In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [13]:
df_1 = pd.read_csv(EXPORT_PATH)
df_1

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,extendido,0.774393,0.352836,-0.055008,0.999588,0.785520,0.329893,-0.044558,0.999092,0.785439,...,-0.071866,0.962630,0.173090,0.723486,0.049177,0.806732,0.149688,0.748092,-0.181912,0.966960
1,extendido,0.775380,0.349040,-0.044754,0.999494,0.786016,0.321030,-0.034638,0.998975,0.785670,...,-0.019392,0.941453,0.171672,0.734957,0.142398,0.799740,0.157518,0.754437,-0.113013,0.941086
2,extendido,0.775654,0.349286,-0.043446,0.999468,0.786195,0.321062,-0.033021,0.998926,0.785845,...,-0.033951,0.940709,0.171914,0.734597,0.138052,0.797228,0.157439,0.754451,-0.134083,0.939878
3,extendido,0.775836,0.349500,-0.043579,0.999443,0.786340,0.321093,-0.033188,0.998875,0.785993,...,-0.045194,0.939181,0.174667,0.734602,0.133422,0.793797,0.157431,0.754582,-0.146636,0.937701
4,extendido,0.775827,0.349868,-0.042317,0.999443,0.786340,0.321220,-0.032107,0.998872,0.785993,...,-0.062718,0.938008,0.176560,0.735710,0.103125,0.790132,0.157436,0.760932,-0.166722,0.935510
5,extendido,0.775711,0.350685,-0.040994,0.999443,0.786328,0.321680,-0.030772,0.998874,0.785985,...,-0.063838,0.937507,0.176355,0.736042,0.098657,0.788516,0.156225,0.763761,-0.168144,0.934468
6,extendido,0.775439,0.353476,-0.045890,0.999432,0.786141,0.324045,-0.035573,0.998864,0.785830,...,-0.050027,0.936484,0.171688,0.733222,0.101923,0.785625,0.156569,0.756508,-0.147255,0.931929
7,extendido,0.773268,0.363228,-0.055173,0.999391,0.785224,0.334708,-0.045530,0.998794,0.785027,...,-0.007216,0.937858,0.180082,0.733684,0.149326,0.791044,0.160325,0.750417,-0.099240,0.931892
8,extendido,0.773523,0.367683,-0.048513,0.999393,0.785547,0.339464,-0.037574,0.998806,0.785342,...,-0.042417,0.936297,0.170960,0.733018,0.080861,0.786946,0.156008,0.750200,-0.139131,0.929742
9,extendido,0.780720,0.407582,-0.042723,0.999387,0.793168,0.382309,-0.029897,0.998824,0.793016,...,-0.040838,0.931720,0.175793,0.735178,0.101979,0.791222,0.149848,0.758398,-0.136893,0.925390


In [22]:
df_2 = pd.read_csv(EXPORT_PATH)
df_2

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,extendido,0.784265,0.296285,-0.061544,0.999939,0.788355,0.269168,-0.053095,0.999866,0.787655,...,-0.051317,0.968681,0.164509,0.687285,0.246771,0.863416,0.097396,0.749445,-0.144762,0.970559
1,flexionado,0.788569,0.703924,0.030560,0.999904,0.803537,0.688756,0.047114,0.999839,0.804244,...,-0.148260,0.975720,0.136344,0.697353,0.177238,0.812613,0.076903,0.757497,-0.236108,0.978667
2,flexionado,0.793274,0.691920,0.020039,0.999894,0.806514,0.673402,0.036763,0.999843,0.806771,...,-0.201220,0.977817,0.140743,0.696921,0.175239,0.802652,0.074866,0.752592,-0.296906,0.982657
3,flexionado,0.794055,0.679369,0.015844,0.999891,0.807175,0.658903,0.033597,0.999837,0.807397,...,-0.200838,0.978067,0.137881,0.696932,0.169881,0.806020,0.074853,0.752637,-0.298156,0.983072
4,flexionado,0.795486,0.657481,0.013757,0.999897,0.808144,0.633657,0.031821,0.999845,0.808257,...,-0.230766,0.979160,0.140125,0.696790,0.228475,0.814694,0.074865,0.752834,-0.332339,0.984071
5,flexionado,0.797271,0.575231,-0.024375,0.999900,0.809785,0.552483,-0.011916,0.999838,0.809825,...,-0.079644,0.981553,0.148406,0.679243,0.274971,0.843933,0.082550,0.749159,-0.164668,0.986176
6,flexionado,0.798400,0.559001,-0.029526,0.999898,0.810928,0.536823,-0.017431,0.999829,0.810721,...,-0.036665,0.981996,0.148146,0.678263,0.277624,0.852194,0.082329,0.748492,-0.117851,0.986455
7,extendido,0.788984,0.344436,-0.050263,0.999947,0.799217,0.317417,-0.042935,0.999897,0.798688,...,-0.003860,0.979295,0.157488,0.678313,0.213809,0.901527,0.103661,0.741908,-0.089647,0.981472
8,extendido,0.786597,0.351131,-0.067464,0.999963,0.798454,0.325065,-0.060232,0.999918,0.798189,...,-0.070510,0.973484,0.167651,0.675434,0.282128,0.898883,0.107732,0.744673,-0.163476,0.976827
9,extendido,0.786621,0.351082,-0.063151,0.999964,0.798460,0.324988,-0.056179,0.999918,0.798194,...,-0.084760,0.973408,0.167891,0.675600,0.302128,0.899001,0.107191,0.745145,-0.178807,0.976880


In [23]:
df = pd.concat([df_1,df_2], axis=0)
df

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,extendido,0.774393,0.352836,-0.055008,0.999588,0.785520,0.329893,-0.044558,0.999092,0.785439,...,-0.071866,0.962630,0.173090,0.723486,0.049177,0.806732,0.149688,0.748092,-0.181912,0.966960
1,extendido,0.775380,0.349040,-0.044754,0.999494,0.786016,0.321030,-0.034638,0.998975,0.785670,...,-0.019392,0.941453,0.171672,0.734957,0.142398,0.799740,0.157518,0.754437,-0.113013,0.941086
2,extendido,0.775654,0.349286,-0.043446,0.999468,0.786195,0.321062,-0.033021,0.998926,0.785845,...,-0.033951,0.940709,0.171914,0.734597,0.138052,0.797228,0.157439,0.754451,-0.134083,0.939878
3,extendido,0.775836,0.349500,-0.043579,0.999443,0.786340,0.321093,-0.033188,0.998875,0.785993,...,-0.045194,0.939181,0.174667,0.734602,0.133422,0.793797,0.157431,0.754582,-0.146636,0.937701
4,extendido,0.775827,0.349868,-0.042317,0.999443,0.786340,0.321220,-0.032107,0.998872,0.785993,...,-0.062718,0.938008,0.176560,0.735710,0.103125,0.790132,0.157436,0.760932,-0.166722,0.935510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30,extendido,0.782529,0.309496,-0.060305,0.999941,0.792464,0.280140,-0.053839,0.999880,0.792095,...,-0.080801,0.981789,0.153336,0.675442,0.272803,0.905225,0.103865,0.745392,-0.174234,0.983200
31,extendido,0.763239,0.289211,-0.174875,0.999962,0.775721,0.263912,-0.160021,0.999909,0.776172,...,-0.102476,0.981494,0.166905,0.694042,0.269436,0.904701,0.100278,0.750937,-0.193645,0.983886
32,extendido,0.762119,0.286649,-0.176033,0.999963,0.774738,0.261487,-0.161142,0.999911,0.775208,...,-0.102772,0.981622,0.167455,0.690118,0.270631,0.904350,0.100228,0.749384,-0.194667,0.984107
33,extendido,0.760296,0.283310,-0.200601,0.999965,0.773082,0.259244,-0.181967,0.999915,0.773696,...,-0.127937,0.981767,0.167545,0.684581,0.325274,0.903146,0.100057,0.746970,-0.225987,0.984338


In [76]:
df.head()

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
0,neutro,0.502405,0.115980,-0.126397,0.999031,0.509151,0.097383,-0.149408,0.999170,0.510624,...,0.352641,0.581703,0.513850,0.913589,0.024361,0.948712,0.539293,0.855410,0.303940,0.677512
1,flexionado,0.472600,0.388244,-0.068190,0.999933,0.481589,0.377964,-0.085519,0.999957,0.483421,...,0.626295,0.556920,0.520238,0.939118,-0.124597,0.954366,0.554266,0.817905,0.649184,0.493745
2,flexionado,0.472633,0.376321,-0.068483,0.999933,0.481666,0.366803,-0.087626,0.999958,0.483499,...,0.621485,0.545250,0.518716,0.940216,-0.114504,0.955483,0.553697,0.818930,0.645266,0.484176
3,flexionado,0.472675,0.370474,-0.056999,0.999934,0.481674,0.358771,-0.078701,0.999960,0.483502,...,0.593467,0.542811,0.518067,0.940524,-0.116002,0.956998,0.552629,0.819090,0.607894,0.483393
4,flexionado,0.472750,0.354556,-0.040285,0.999935,0.481694,0.343744,-0.063144,0.999961,0.483516,...,0.562967,0.546462,0.517828,0.938163,-0.085412,0.957767,0.552654,0.846781,0.576607,0.487331


In [17]:
df.tail()

NameError: name 'df' is not defined

In [78]:
df[df['class']=='extendido']

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z31,v31,x32,y32,z32,v32,x33,y33,z33,v33
5,extendido,0.502610,0.140351,-0.067839,0.999252,0.508379,0.122076,-0.088989,0.999391,0.509688,...,0.361801,0.627713,0.514552,0.934133,-0.158818,0.975754,0.537099,0.868836,0.309454,0.653462
6,extendido,0.506286,0.137658,-0.079192,0.998978,0.511973,0.117781,-0.101628,0.999111,0.513256,...,0.257583,0.578696,0.518224,0.932504,-0.049218,0.944847,0.535459,0.898081,0.199482,0.662440
7,extendido,0.506202,0.136903,-0.094477,0.999016,0.511920,0.117213,-0.117343,0.999143,0.513206,...,0.287624,0.582171,0.517998,0.932446,-0.062795,0.946891,0.536338,0.888425,0.230290,0.668297
8,extendido,0.506133,0.137021,-0.083607,0.999057,0.511889,0.117328,-0.106080,0.999172,0.513185,...,0.274915,0.583098,0.516750,0.932320,-0.058346,0.948314,0.536248,0.886373,0.220431,0.671329
9,extendido,0.506003,0.137115,-0.084911,0.999092,0.511701,0.117351,-0.107181,0.999199,0.512998,...,0.284183,0.587038,0.516725,0.931722,-0.050640,0.950035,0.537174,0.882327,0.240841,0.676383
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43,extendido,0.552845,0.088871,-0.103061,0.998514,0.560574,0.074314,-0.121032,0.999384,0.562332,...,0.340988,0.628272,0.560331,0.914102,-0.200458,0.976142,0.562450,0.873955,0.291883,0.725909
44,extendido,0.551624,0.088326,-0.096769,0.998659,0.559341,0.073959,-0.118640,0.999444,0.560977,...,0.400457,0.633687,0.560365,0.914758,-0.187586,0.977932,0.562333,0.873041,0.353537,0.733165
45,extendido,0.550277,0.088299,-0.093956,0.998719,0.557887,0.073954,-0.117077,0.999467,0.559408,...,0.412065,0.635274,0.560308,0.915411,-0.180799,0.978788,0.562333,0.873422,0.367332,0.736709
46,extendido,0.549287,0.088863,-0.098310,0.998771,0.556492,0.074404,-0.121223,0.999487,0.557967,...,0.428948,0.634794,0.560184,0.915372,-0.200569,0.979610,0.562767,0.871764,0.387001,0.737292


In [24]:
X = df.drop('class', axis=1) # features
y = df['class'] # target

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

## Entrenar con el modelo de clasificación de Machine Learning


In [26]:
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import StandardScaler 

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [27]:
pipelines = {
    #'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [28]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [29]:
fit_models['rc'].predict(X_test)

array(['extendido', 'extendido', 'extendido', 'flexionado', 'extendido',
       'flexionado', 'flexionado', 'extendido', 'extendido', 'flexionado',
       'flexionado', 'extendido', 'extendido', 'flexionado', 'flexionado',
       'extendido', 'extendido', 'flexionado', 'extendido', 'flexionado',
       'flexionado'], dtype='<U10')

## Evaluar y Serializar el modelo


In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score # Accuracy metrics 
import pickle 

In [31]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat),
          precision_score(y_test.values, yhat, average="weighted", pos_label="derecha"),
          recall_score(y_test.values, yhat, average="weighted", pos_label="derecha"))

rc 1.0 1.0 1.0
rf 1.0 1.0 1.0
gb 1.0 1.0 1.0


c:\Users\Felipe\miniconda3\envs\env_humai_estudiantes\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'derecha') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Felipe\miniconda3\envs\env_humai_estudiantes\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'derecha') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Felipe\miniconda3\envs\env_humai_estudiantes\lib\site-packages\sklearn\metrics\_classification.py:1396: UserWarning: Note that pos_label (set to 'derecha') is ignored when average != 'binary' (got 'weighted'). You may use labels=[pos_label] to specify a single positive class.
  warnings.warn(
c:\Users\Felipe\miniconda3\envs\env_humai_estudiantes\lib\site-packages\sklearn\metrics\_cl

In [32]:
yhat = fit_models['rf'].predict(X_test)

In [33]:
yhat[:10]

array(['extendido', 'extendido', 'extendido', 'flexionado', 'extendido',
       'flexionado', 'flexionado', 'extendido', 'extendido', 'flexionado'],
      dtype=object)

In [89]:
with open(MODEL_PATH, 'wb') as f:
    pickle.dump(fit_models['rf'], f)

# Detectar con el modelo


In [34]:
with open(MODEL_PATH, 'rb') as f:
    model = pickle.load(f)

FileNotFoundError: [Errno 2] No such file or directory: 'push_ups.pkl'

In [43]:
# X = pd.DataFrame([row], columns=landmarks[1:])

In [44]:
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
current_stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Detección de Pose
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Extract landmarks
        try:
            
            row = np.array([[res.x, res.y, res.z, res.visibility] for res in results.pose_landmarks.landmark]).flatten().tolist()
            X = pd.DataFrame([row], columns=landmarks[1:])
            
            body_language_class = model.predict(X)[0]
            
            body_language_prob = model.predict_proba(X)[0]
            print(body_language_class, body_language_prob)
            
            if body_language_class or body_language_prob:
                print('predicted')
            else :
                print('prediction error')

            if body_language_class == "derecha" and body_language_prob[body_language_prob.argmax()] >= 0.7:
                current_stage = "derecha"
            elif current_stage=="derecha" and body_language_class=="izquierda" and body_language_prob[body_language_prob.argmax()] >= 0.7:
                current_stage = "izquierda"
                counter +=1
                
            # obtener el estado del box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Mostrar las clases
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, body_language_class.split(' ')[0]
                        , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Mostrar Probabilidad
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(body_language_prob[np.argmax(body_language_prob)],2))
                       , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
            
            # Mostrar Conteo
            cv2.putText(image, 'COUNT'
                        , (180,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter)
                        , (175,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
  
        except Exception as e:
            pass

        cv2.imshow('En vivo sin procesar', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

neutro [0.09 0.   0.91]
predicted
neutro [0.19 0.   0.81]
predicted
neutro [0.2 0.  0.8]
predicted
neutro [0.19 0.   0.81]
predicted
neutro [0.19 0.   0.81]
predicted
neutro [0.19 0.   0.81]
predicted
neutro [0.19 0.   0.81]
predicted
neutro [0.19 0.   0.81]
predicted
neutro [0.36 0.02 0.62]
predicted
neutro [0.21 0.   0.79]
predicted
neutro [0.38 0.02 0.6 ]
predicted
neutro [0.36 0.02 0.62]
predicted
neutro [0.19 0.   0.81]
predicted
neutro [0.1 0.  0.9]
predicted
neutro [0.05 0.08 0.87]
predicted
neutro [0.15 0.04 0.81]
predicted
neutro [0.05 0.   0.95]
predicted
neutro [0.05 0.   0.95]
predicted
neutro [0.05 0.   0.95]
predicted
neutro [0.05 0.04 0.91]
predicted
neutro [0.05 0.   0.95]
predicted
neutro [0.05 0.   0.95]
predicted
neutro [0.05 0.   0.95]
predicted
neutro [0.15 0.04 0.81]
predicted
neutro [0.14 0.03 0.83]
predicted
neutro [0.16 0.03 0.81]
predicted
neutro [0.23 0.04 0.73]
predicted
neutro [0.23 0.02 0.75]
predicted
neutro [0.23 0.02 0.75]
predicted
neutro [0.22 0.03 0.

In [60]:
# PRUEBA DE CAMARA
cap = cv2.VideoCapture(0)

# Curl counter variables
counter = 0 
current_stage = None

## Setup mediapipe instance
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()
        
        # Recolor image to RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
      
        # Make detection
        results = pose.process(image)
    
        # Recolor back to BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        # Pose Detections
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS, 
                                 mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4),
                                 mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                                 )
        
        # Extract landmarks
        try:
                       
            # obtener el estado del box
            cv2.rectangle(image, (0,0), (250, 60), (245, 117, 16), -1)
            
            # Mostrar las clases
            cv2.putText(image, 'CLASS'
                        , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
                        
            # Mostrar Probabilidad
            cv2.putText(image, 'PROB'
                        , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, '10.05'
                       , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
                        
            # Mostrar Conteo
            cv2.putText(image, 'COUNT'
                        , (180,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            
        except Exception as e:
            pass

        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()